In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import json
import math
import csv
import pandas as pd
import xlsxwriter

Material = "Material E"

#%% Calculate Core Loss
def core_loss(data_B, data_F, data_T):
    
    DataF1 = data_F
    DataB1 = data_B
    DataT1 = data_T 
    %run MasterCode_DataSplit2.ipynb 

    print('\nPerforming triangular data testing')     
    %run Trig_Tests.ipynb

    print('\nPerforming sinusoidal data testing')
    %run Sine_Tests.ipynb

    print('\nPerforming trapezoidal data testing')
    %run Trap_Tests.ipynb

    Ploss = np.concatenate((sine_test_out,trig_test_out,trap_test_out), axis=0)
    Ind = np.concatenate((sine_index,trig_index,trap_index), axis=0)

    df = pd.concat([pd.DataFrame(Ploss), pd.DataFrame(Ind)], ignore_index=True, axis=1)
    data = df.sort_values(by=1) 
    data_P = data[0].to_numpy()
    data_Ind = data[1].to_numpy()

    for i in range(0,len(data_F)):
        if (data_Ind[i]!=(i+1)):
            data_Ind = np.insert(data_Ind,i,i+1)
            data_P = np.insert(data_P,i,0)

    f1 = './IISc/Result/Volumetric_Loss_'+Material+'.xlsx'

    workbook = xlsxwriter.Workbook(f1)
    worksheet = workbook.add_worksheet('Data')
    row = 0 
    for kk in range (0,len(data_P)):
        worksheet.write(row,0,data_P[kk])
        row +=1
    workbook.close()

    print('Model inference is finished!')
    
    return
    
if __name__ == "__main__":
    dataB = pd.read_csv('./Testing/'+ Material+'/B_Field.csv',header=None)
    dataF = pd.read_csv('./Testing/'+ Material+'/Frequency.csv',header=None)
    dataT = pd.read_csv('./Testing/'+ Material+'/Temperature.csv',header=None)

    data_B = dataB.to_numpy()
    data_F = dataF.to_numpy()
    data_T = dataT.to_numpy()

    # Reproducibility
    random.seed(1)
    np.random.seed(1)
    core_loss(data_B, data_F, data_T)
    
# # End